In [2]:
import osmnx as ox
import networkx as nx
import joblib
import numpy as np
import random
import folium
from folium import plugins
import warnings
from geopy.geocoders import Nominatim

# Ignore all warnings
warnings.filterwarnings("ignore")

# Load the trained ML model
model = joblib.load('traffic_route_model.pkl')

# Function to get the latitude and longitude of a city using geopy
def get_coordinates(city_name):
    geolocator = Nominatim(user_agent="traffic_route_planner")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        print(f"Could not find coordinates for {city_name}")
        return None, None

# Function to fetch the route length from source to destination
def get_route_length(source_city, dest_city):
    # Fetch coordinates for source and destination cities
    source_coords = get_coordinates(source_city)
    dest_coords = get_coordinates(dest_city)
    
    if not source_coords or not dest_coords:
        return 0  # Return 0 if coordinates are not found
    
    print(f"Fetching route data for {source_ity} to {dest_city}...")
    # Fetch the graph data for the source and destination cities using osmnx
    G = ox.graph_from_place(source_city, network_type='drive')
    
    # Find the nearest nodes to the source and destination citiecs
    orig_node = ox.distance.nearest_nodes(G, X=source_coords[1], Y=source_coords[0])
    dest_node = ox.distance.nearest_nodes(G, X=dest_coords[1], Y=dest_coords[0])
    
    # Get the shortest path based on distance
    route = nx.shortest_path(G, orig_node, dest_node, weight='length')
    
    # Calculate the route length in kilometers (using 'length' attribute in meters, divide by 1000 for km)
    route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length')) / 1000  # in km
    
    # Plot the route on an interactive map using folium
    plot_route_on_map(G, route, source_city, dest_city)  # Call function to plot map
    
    return route_length

# Function to plot the route on an interactive folium map
def plot_route_on_map(G, route, source_city, dest_city):
    # Get the coordinates of the source and destination nodes
    orig_node = route[0]
    dest_node = route[-1]
    
    orig_coords = (G.nodes[orig_node]['y'], G.nodes[orig_node]['x'])
    dest_coords = (G.nodes[dest_node]['y'], G.nodes[dest_node]['x'])
    
    # Create a folium map centered around the source city
    m = folium.Map(location=orig_coords, zoom_start=13, control_scale=True)
    
    # Add a marker for the source city
    folium.Marker(location=orig_coords, popup=f'Start: {source_city}', icon=folium.Icon(color='green')).add_to(m)
    
    # Add a marker for the destination city
    folium.Marker(location=dest_coords, popup=f'Destination: {dest_city}', icon=folium.Icon(color='red')).add_to(m)
    
    # Add the route to the map
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(route_coords, color="blue", weight=4, opacity=0.7).add_to(m)
    
    # Add some traffic visual effects (Optional, can be enhanced later)
    folium.plugins.HeatMap(route_coords).add_to(m)
    
    # Save the map to an HTML file and open it in the browser
    m.save("route_map.html")
    print("Map saved as 'route_map.html'. Open this file in your browser to view the route.")

# Function to simulate traffic condition (could be updated to use real-time data via API)
def get_traffic_condition():
    # For simplicity, we're randomly selecting traffic conditions, you can integrate real-time APIs
    traffic_condition = random.randint(1, 7)  # Random traffic condition between 1 and 7
    return traffic_condition

# Main function to predict travel time
def predict_travel_time(source_city, dest_city, time_of_day):
    # Get the route length between source and destination cities
    route_length = get_route_length(source_city, dest_city)
    
    # Get the traffic condition (1-7)
    traffic_condition = get_traffic_condition()
    
    # Prepare the input data for the model (route length, traffic condition, and time of day)
    input_data = np.array([[route_length, traffic_condition, time_of_day]])
    
    # Predict travel time using the model
    predicted_time = model.predict(input_data)
    
    # Return both the predicted travel time and route length
    return predicted_time[0], route_length

# Interactive input for source, destination, and time of day
def main():
    print("=== Traffic Route Planner ===")
    
    # User inputs source and destination cities
    source_city = input("Enter source city: ")
    dest_city = input("Enter destination city: ")
    
    # User input for time of day (0-23)
    time_of_day = int(input("Enter time of day (0-23, in hours): "))
    
    # Get the predicted travel time and route length based on user input
    predicted_travel_time, routedi_length = predict_travel_time(source_city, dest_city, time_of_day)
    
    # Output the predicted travel time and route distance
    print("---Route---")
    print(f"Distance: {route_length:.2f} km")
    print(f"Predicted Travel Time: {predicted_travel_time:.2f} minutes")
    print("A map of the route has been saved as 'route_map.html'. Open this file in your browser.")

# Run the program
if __name__ == "__main__":
    main()


=== Traffic Route Planner ===


Enter source city:  Chennai, Tamilnadu, India
Enter destination city:  Kanchipuram, Tamilnadu, India
Enter time of day (0-23, in hours):  8


Fetching route data for Chennai, Tamilnadu, India to Kanchipuram, Tamilnadu, India...
Map saved as 'route_map.html'. Open this file in your browser to view the route.
---Route---
Distance: 23.56 km
Predicted Travel Time: 65.21 minutes
A map of the route has been saved as 'route_map.html'. Open this file in your browser.
